In [1]:
import os

In [2]:
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/lavish./Desktop/Deep Learning/End-to-End-Chest-cancer-Classification-Using-MLFLOW-DVC'

## Update the Config

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data : Path
    params_image_size : list
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool



## Update the Configuration Manager in src

In [6]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

## Update the components

In [8]:
import tensorflow as tf
from pathlib import Path

In [9]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from pathlib import Path

class Training:
    def __init__(self, config):
        """
        Initializes the Training class with the provided configuration.
        
        Args:
            config (TrainingConfig): Configuration object containing paths and training parameters.
        """
        self.config = config

    def get_base_model(self):
        """
        Loads the base model from the updated base model path and applies L2 regularization 
        and Dropout to the layers to help prevent overfitting. Recompiles the model with a 
        reduced learning rate.
        """
        # Load the base model
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Apply L2 regularization to layers if applicable
        for layer in self.model.layers:
            if hasattr(layer, 'kernel_regularizer'):
                layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)  # L2 Regularization

        # Recompile the model with a lower learning rate
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Reduce the learning rate
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        """
        Sets up data generators for training and validation. 
        Applies data augmentation to the training data if enabled, 
        while validation data remains unaugmented.
        """
        # Common generator arguments
        datagenerator_kwargs = dict(
            rescale=1./255,          # Rescale images
            validation_split=0.20     # Split for validation (20%)
        )

        # Dataflow options
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Image size without channels
            batch_size=self.config.params_batch_size,        # Batch size
            interpolation="bilinear"                        # Interpolation method
        )

        # Validation data generator (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Validation generator setup
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Apply data augmentation to the training data if enabled
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,          # Random rotation
                horizontal_flip=True,       # Random horizontal flip
                width_shift_range=0.2,      # Horizontal shift
                height_shift_range=0.2,     # Vertical shift
                shear_range=0.2,            # Shear transformation
                zoom_range=0.2,             # Random zoom
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator  # No augmentation for training

        # Training generator setup
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the trained model to the specified path.
        
        Args:
            path (Path): File path to save the model.
            model (tf.keras.Model): Trained model to save.
        """
        model.save(path)

    def train(self):
        """
        Trains the model using the training and validation generators. 
        Includes early stopping and learning rate reduction as callbacks to 
        prevent overfitting and improve performance.
        """
        # Define steps per epoch and validation steps
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Define callbacks to prevent overfitting and adjust learning rate
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[early_stopping, reduce_lr]  # Apply callbacks
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

## update the pipeline

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-10-24 11:24:41,105: INFO : common : yaml file: config/config.yaml loaded successfully]
[2024-10-24 11:24:41,107: INFO : common : yaml file: params.yaml loaded successfully]
[2024-10-24 11:24:41,108: INFO : common : created directory at: artifacts]
[2024-10-24 11:24:41,108: INFO : common : created directory at: artifacts/training]
[2024-10-24 11:24:41,245: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


/opt/anaconda3/envs/chest/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5432 - loss: 0.7144

/opt/anaconda3/envs/chest/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.5452 - loss: 0.7114 - val_accuracy: 0.8438 - val_loss: 0.3863 - learning_rate: 1.0000e-04
Epoch 2/8
 1/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.6875 - loss: 0.5138

...

2024-10-24 11:25:19.980019: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7949 - loss: 0.4742 - val_accuracy: 0.9375 - val_loss: 0.2844 - learning_rate: 1.0000e-04
Epoch 4/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.8750 - loss: 0.3409 - val_accuracy: 1.0000 - val_loss: 0.1522 - learning_rate: 1.0000e-04
Epoch 5/8


2024-10-24 11:26:03.361444: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


17/17 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.8325 - loss: 0.4000 - val_accuracy: 0.9844 - val_loss: 0.2233 - learning_rate: 1.0000e-04
Epoch 6/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6667 - loss: 0.3803 - val_accuracy: 1.0000 - val_loss: 0.1041 - learning_rate: 1.0000e-04
Epoch 7/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9241 - loss: 0.3121 - val_accuracy: 0.9844 - val_loss: 0.2076 - learning_rate: 1.0000e-04
Epoch 8/8
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 1.0000 - loss: 0.1802 - val_accuracy: 1.0000 - val_loss: 0.0516 - learning_rate: 1.0000e-04
[2024-10-24 11:27:30,752: WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


2024-10-24 11:27:30.720957: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
